# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://1229fd35c5ba:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-25e7847b-ef7c-4cd1-833b-83c800390d8e;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
# Import your module
from santiagomdeo.spark_utils import SparkUtils

schema_xd = [
    ("src_ip", "int"),
    ("src_port", "int"),
    ("dest_ip", "int"),
    ("dest_port", "int"),
    ("protocol", "int"),
    ("bytes_in", "int"),
    ("bytes_out", "int"),
    ("num_pkts_in", "int"),
    ("num_pkts_out", "int"),
    ("entropy", "float"),
    ("total_entropy", "float"),
    ("mean_ipt", "float"),
    ("time_start", "int"),
    ("time_end", "int"),
    ("duration", "int"),
    ("label", "string")
]

ips_schema = SparkUtils.generate_schema(schema_xd)

ips_df = spark.read.option("header", True).schema(ips_schema).csv("/opt/spark/work-dir/data/ips/input")


int
int
int
int
int
int
int
int
int
float
float
float
int
int
int
string


# Transformations

In [3]:
ips_df.show(n=5)

+------+--------+-------+---------+--------+--------+---------+-----------+------------+---------+-------------+----------+----------+----------+--------+-------+
|src_ip|src_port|dest_ip|dest_port|protocol|bytes_in|bytes_out|num_pkts_in|num_pkts_out|  entropy|total_entropy|  mean_ipt|time_start|  time_end|duration|  label|
+------+--------+-------+---------+--------+--------+---------+-----------+------------+---------+-------------+----------+----------+----------+--------+-------+
|  3170|   29429|   4440|    20361|       6|    4531|     1822|         28|          39|3.2449083|     43.07527| 0.9374156|1609300755|1609300805|      50| benign|
|  4793|   61829|   8333|    64587|      17|    2931|     8838|         27|           4|0.4723278|     8.161594|0.48300773|1603360433|1603360486|      53| benign|
|  6574|   64068|   1182|    25532|       6|    1366|     7889|         47|          29|2.1887074|     9.151795| 0.8613521|1601181873|1601181921|      48| benign|
|  7198|   20004|   34

In [13]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
# Build schema
#source ip
#connected or message: {source port, destination port, protocol}
#destination ip

#table 1, a single coulmn with ips
#table 2 connections
#table 3

from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame


#Vertices DataFrame:
ip_nodes = ips_df.select(
    col("src_ip").alias("ip")
)
ip_nodes2 = ips_df.select(
    col("dest_ip").alias("ip")
)
vertices_df = reduce(DataFrame.unionAll, [ip_nodes, ip_nodes2]).dropDuplicates(["ip"])


#Edges DataFrame:

#creo que hacer 3 connexiones diferentes seria mas divertido
outlier_connections_ip = ips_df.filter(ips_df.label == "outlier").select(
    col("src_ip").alias("src"),
    col("dest_ip").alias("dst"),
    col("src_port"),
    col("dest_port"),
    col("protocol"),
    col("duration")
)
benign_connections_ip = ips_df.filter(ips_df.label == "benign").select(
    col("src_ip").alias("src"),
    col("dest_ip").alias("dst"),
    col("src_port"),
    col("dest_port"),
    col("protocol"),
    col("duration")
)
malicious_connections_ip = ips_df.filter(ips_df.label == "malicious").select(
    col("src_ip").alias("src"),
    col("dest_ip").alias("dst"),
    col("src_port"),
    col("dest_port"),
    col("protocol"),
    col("duration")
)


vertices_df.show(n=4)
outlier_connections_ip.show(n=4)




+----+
|  ip|
+----+
|6466|
|3794|
|2122|
|8638|
+----+
only showing top 4 rows
+----+----+--------+---------+--------+--------+
| src| dst|src_port|dest_port|protocol|duration|
+----+----+--------+---------+--------+--------+
|7198|3474|   20004|    57629|       6|      11|
|4667|3306|    2823|    17262|      17|      49|
| 210|4656|   48815|    54636|       6|      52|
|5937|5014|   38636|    37761|       6|      25|
+----+----+--------+---------+--------+--------+
only showing top 4 rows


# Writing Data in Neo4j

In [ ]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"


vertices_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":IPs") \
  .option("node.keys", "ip") \
  .save()

#took 21 seconds
print(f"{vertices_df.count()} IPs wrote in Neo4j")

# outlier_connections_ip
# benign_connections_ip
# malicious_connections_ip





10000 IPs wrote in Neo4j


In [ ]:
#3 minutos
outlier_connections_ip.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "OUTLIER_CONNECTION") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":IPs") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:ip") \
  .option("relationship.target.labels", ":IPs") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:ip") \
  .save()

#.option("relationship.source.node.keys", "src:id") format seems to take on column1InRelationship:ColumnInEdge

print(f"{outlier_connections_ip.count()} outlier edges wrote in Neo4j")

23647 outlier edges wrote in Neo4j


In [16]:
#minutos
benign_connections_ip.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "BENIGN_CONNECTION") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":IPs") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:ip") \
  .option("relationship.target.labels", ":IPs") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:ip") \
  .save()

#.option("relationship.source.node.keys", "src:id") format seems to take on column1InRelationship:ColumnInEdge

print(f"{benign_connections_ip.count()} benign edges wrote in Neo4j")

51294 benign edges wrote in Neo4j


In [17]:
malicious_connections_ip.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "MALICIOUS_CONNECTION") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":IPs") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:ip") \
  .option("relationship.target.labels", ":IPs") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:ip") \
  .save()

#.option("relationship.source.node.keys", "src:id") format seems to take on column1InRelationship:ColumnInEdge

print(f"{malicious_connections_ip.count()} malicious edges wrote in Neo4j")

25059 malicious edges wrote in Neo4j


# Read and Query Graphs with PySpark

In [20]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 

cypher_df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            match (n:IPs{ip: 2659})-[r]->(m)
            return n.ip, r, m.ip
            """) \
    .load()

cypher_df.show(truncate=False)

+----+-------------------------------------------------------------+----+
|n.ip|r                                                            |m.ip|
+----+-------------------------------------------------------------+----+
|2659|{99103, MALICIOUS_CONNECTION, 18, 5299, 26, 17473, 7587, 6}  |3157|
|2659|{91446, MALICIOUS_CONNECTION, 18, 9492, 24, 48039, 54369, 17}|3221|
|2659|{75428, MALICIOUS_CONNECTION, 18, 712, 59, 9069, 30195, 6}   |436 |
|2659|{65200, BENIGN_CONNECTION, 18, 9597, 21, 53974, 5253, 17}    |6998|
|2659|{64969, BENIGN_CONNECTION, 18, 6685, 49, 41738, 3583, 6}     |73  |
|2659|{58039, BENIGN_CONNECTION, 18, 2129, 9, 18071, 9968, 6}      |259 |
|2659|{60525, BENIGN_CONNECTION, 18, 1214, 60, 58713, 21988, 17}   |2939|
|2659|{48581, BENIGN_CONNECTION, 18, 9494, 16, 37973, 19994, 6}    |551 |
|2659|{7402, OUTLIER_CONNECTION, 18, 8775, 27, 32649, 49840, 6}    |4261|
+----+-------------------------------------------------------------+----+



In [ ]:
sc.stop()